In [1]:
%pylab
import numpy as np
import nibabel as nib

from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table

import dipy.reconst.dti as dti

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


Load files

In [2]:
subj = 'F22_Adult'

dname = 'E:/Celine/Ferret_b2n/'+subj+'/'

fdwi = subj+'_rot_den' #nii from b2n   f25-p2-noswap-noflip
niigz = '.nii.gz'

img = nib.load(dname+fdwi+niigz)
data = img.get_data()
affine = img.affine

fbval = 'bvals' 
fbvec = 'bvecs'

bvals, bvecs = read_bvals_bvecs(dname+fbval, dname+fbvec)

gtab = gradient_table(bvals, bvecs, b0_threshold=50)

fmask = '' #dname + 'mask_to_dwi.nii.gz' 

Adjust gradients

In [3]:
non0bvecs = np.count_nonzero(bvecs[:,0])

In [8]:
affine

array([[ 0.        ,  0.23999999,  0.        ,  0.        ],
       [-0.23999999,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.23999999,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [9]:
affine_tricked = affine[:,[2,1,0,3]]  #swap 1st and 3rd axis 2103

In [10]:
affine_tricked[:,0] *= -1  # flip 1st and 2nd axis
affine_tricked[:,1] *= -1
affine_tricked[:,2] *= 1

In [11]:
affine_again = np.tile([affine_tricked],(non0bvecs,1,1)) #affine_tricked

In [12]:
from dipy.core.gradients import reorient_bvecs
gtab_new = reorient_bvecs(gtab,affine_again)

In [13]:
affine_tricked

array([[-0.        , -0.23999999,  0.        ,  0.        ],
       [-0.        , -0.        , -0.23999999,  0.        ],
       [-0.23999999, -0.        ,  0.        ,  0.        ],
       [-0.        , -0.        ,  0.        ,  1.        ]])

Load or create mask

In [14]:
try:
    from dipy.segment.mask import applymask
    mask_img = nib.load(fmask)
    mask = mask_img.get_data()
    mask[mask>1] =1 #binarize
    maskdata = applymask(data,mask)
except IOError:
    print "Cannot open ",fmask,"\n will generate an automated mask using median_otsu from Dipy"

    from dipy.segment.mask import median_otsu
    maskdata, mask = median_otsu(data, 3, 1, False, dilate=2 ) #crop:removes the contours, changes coordinates

C:\Users\cdelettre\Anaconda2\lib\site-packages\skimage\filter\__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


Cannot open   
 will generate an automated mask using median_otsu from Dipy


Create and fit tensor model (DTI)

In [15]:
tenmodel = dti.TensorModel(gtab_new)  ### gtab_new if gradients modified

In [16]:
tenfit = tenmodel.fit(maskdata)

Compute FA, MD, RGB

In [17]:
#print('Computing anisotropy measures (FA, MD, RGB)')
from dipy.reconst.dti import fractional_anisotropy, color_fa, lower_triangular

FA = fractional_anisotropy(tenfit.evals)

In [18]:
FA[np.isnan(FA)] = 0

In [19]:
fa_img = nib.Nifti1Image(FA.astype(np.float32), affine) #img.affine
nib.save(fa_img, dname+'tensor_fa_'+fdwi+"_modifiedgtab_-2-10"+niigz)

In [98]:
#MD1 = dti.mean_diffusivity(tenfit.evals)
#nib.save(nib.Nifti1Image(MD1.astype(np.float32), affine), dname+'tensors_md.nii.gz')

In [99]:
#FA = np.clip(FA, 0, 1)
#RGB = color_fa(FA, tenfit.evecs)
#nib.save(nib.Nifti1Image(np.array(255 * RGB, 'uint8'), affine), dname+'tensor_rgb.nii.gz')

Tracking

In [20]:
from dipy.direction import peaks_from_model
from dipy.data import default_sphere
from dipy.data import get_sphere

peaks_sphere = get_sphere('symmetric362')

ten_peaks = peaks_from_model(tenmodel, data, peaks_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=30,
                             mask=mask)

In [21]:
from dipy.tracking.local import ThresholdTissueClassifier

stopping_thr = 0.1

classifier = ThresholdTissueClassifier(FA, stopping_thr)

Set tracking seed region

In [22]:
from dipy.tracking import utils

seed_density = 2

seed_mask = FA > 0.1 #0.4 

seeds = utils.seeds_from_mask(
    seed_mask,
    density=seed_density,
    affine=affine)

In [23]:
from dipy.tracking.local import LocalTracking

streamlines = LocalTracking(ten_peaks,classifier,seeds,affine,step_size=0.12,return_all=True) #return_all False, returns only the streamlines that reach another point (no streamline with a length of 0)
streamlines = list(streamlines)

In [24]:
from dipy.io.trackvis import save_trk

save_trk(dname+"tracto_"+fdwi+"_modifiedgtab_-2-10.trk", streamlines, affine, FA.shape)

In [25]:
#advised by Saber, not working atm
from nibabel.streamlines import save
from nibabel.streamlines import Tractogram

save(Tractogram(streamlines,affine),dname+"tracto_gtab_tricked_den_sd_2_peaks08.trk")

ValueError: dictionary update sequence element #0 has length 4; 2 is required